In [13]:
import requests
import os
import datetime
import pandas as pd
from dotenv import load_dotenv

In [5]:
# Load variables from .env
load_dotenv()

# Read from environment
access_url = os.getenv("SIMPLEFIN_ACCESS_TOKEN")

# Display error if token not found
if access_url is None:
    raise ValueError("SimpleFIN Access Token was not found in environment.")


In [6]:
# Parse authentication from access URL
scheme, rest = access_url.split("//", 1)
auth, rest = rest.split("@", 1)

username, password = auth.split(":", 1)

url = scheme + "//" + rest + "/accounts"

In [7]:
# Fetch data from SimpleFIN
response = requests.get(url, auth=(username, password))

# Display error is we fail to feth the accounts
if response.status_code != 200:
    raise Exception(f"Failed to fetch accounts: {response.status_code} - {response.text}")

data = response.json()

In [10]:
# Helper function to convert timestamps into datetime
def ts_to_datetime(ts):
    return datetime.datetime.fromtimestamp(ts)

transactions = []

# Account name and balance for all accounts
for account in data["accounts"]:
    acct_name = account["name"]
    balance_date = ts_to_datetime(account["balance-date"])

    # Info about each transaction within each account
    for trans in account.get("transactions", []):
        transactions.append({
            "account": acct_name,
            "date": ts_to_datetime(trans["posted"]).date(),
            "description": trans["description"],
            "amount": trans["amount"],
            "category": trans.get("category", None)
        })

In [14]:
# Export to csv

df = pd.DataFrame(transactions).sort_values(by="date", ascending=True)

today = datetime.datetime.now().strftime('%Y-%m-%d')
filename = f"transactions_{today}.csv"
df.to_csv(filename, index=False)

# Confirmation message
print(f" Successfully exported {len(df)} transactions to {filename}")

 Successfully exported 2 transactions to transactions_2025-06-05.csv
